Script para obtener estadísticas de los corpus

In [1]:
# Cargamos las librerías necesarias
import numpy as np
import pandas as pd
import re
import os, glob
from sklearn.datasets import load_files

# Habilitamos display interactivo y enlace con nuestra cuenta de google
from google.colab import data_table
from google.colab import drive

In [2]:
# Enlazamos nuestro notebook en Colab con nuestro almacenamiento en Google Drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Cargamos el conjunto de entrenamiento leyendo todas las notas. Sin tokenizar

In [3]:
#MOD
DATA_DIR_TRAIN = "/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/"
data = load_files(DATA_DIR_TRAIN, encoding="utf-8", decode_error="replace", shuffle=False)
data_train = pd.DataFrame(data.data, columns = ['texto'])
data_train['fichero'] = data.filenames
data_train

,texto,fichero
0,T1\tSITUACION_LABORAL 352 369\tsiguió trabajan...,/content/drive/MyDrive/Colab Notebooks/Corpus/...
1,"﻿El 21 de enero de 2020, un hombre de 50 años ...",/content/drive/MyDrive/Colab Notebooks/Corpus/...
2,T1\tPROFESION 4689 4713\tprofesionales sanitar...,/content/drive/MyDrive/Colab Notebooks/Corpus/...
3,"El 2 de febrero de 2020, una mujer de 28 años ...",/content/drive/MyDrive/Colab Notebooks/Corpus/...
4,T1\tSITUACION_LABORAL 628 651\thabía estado tr...,/content/drive/MyDrive/Colab Notebooks/Corpus/...
...,...,...
1055,Paciente de 35 años que acude a urgencias porq...,/content/drive/MyDrive/Colab Notebooks/Corpus/...
1056,T2\tPACIENTE 207 227\ttrabajador del campo\nT4...,/content/drive/MyDrive/Colab Notebooks/Corpus/...
1057,Varón de 83 años de edad con antecedentes pers...,/content/drive/MyDrive/Colab Notebooks/Corpus/...
1058,T2\tPACIENTE 105 135\tlicenciada en ciencias e...,/content/drive/MyDrive/Colab Notebooks/Corpus/...


In [4]:
# Nos quedamos únicamente con los ficheros de estudio en base a como se organiza el corpus en carpetas
data_train = data_train[(data_train.fichero.str.contains('.txt', case=False, regex=False)) & (data_train.fichero.str.contains('task1', case=False, regex=False))]

#df["deteccion"] = df['texto'].str.contains('trabaj|ocupacion|profesion', case=False, regex=True)

#Eliminamos la ruta para comparar el nombre de las notas a revisar con las presentes en el conjunto de test de MEDDOPROF
data_train['fichero'] = data_train["fichero"].apply(lambda x: os.path.splitext(os.path.basename(x))[0])
data_train.head(5)

<ipython-input-4-c26fd6f6aabc>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train['fichero'] = data_train["fichero"].apply(lambda x: os.path.splitext(os.path.basename(x))[0])


,texto,fichero
1,"﻿El 21 de enero de 2020, un hombre de 50 años ...",32085846_ES
3,"El 2 de febrero de 2020, una mujer de 28 años ...",32119083_ES
5,Ingresó en nuestro hospital un hombre de 23 añ...,32168162_ES
7,Nuestro paciente es un hombre de 56 años con i...,32222713_ES
9,"﻿Una niña de 5 semanas, nacida a término, con ...",32318729_ES


In [5]:
# Vemos la longitud de train
len(data_train)

265

In [6]:
data_train

,texto,fichero
1,"﻿El 21 de enero de 2020, un hombre de 50 años ...",32085846_ES
3,"El 2 de febrero de 2020, una mujer de 28 años ...",32119083_ES
5,Ingresó en nuestro hospital un hombre de 23 añ...,32168162_ES
7,Nuestro paciente es un hombre de 56 años con i...,32222713_ES
9,"﻿Una niña de 5 semanas, nacida a término, con ...",32318729_ES
...,...,...
521,"Apicultor de 58 años, sin antecedentes de inte...",cc_reumatologia52
523,Hombre de 57 años que acude al servicio de urg...,cc_reumatologia76
525,Paciente de 35 años que acude a urgencias porq...,cc_reumatologia77
527,Varón de 83 años de edad con antecedentes pers...,cc_reumatologia88


Cogemos la versión tokenizada por spaCy del conjunto de train y test

In [7]:
# MOD
ruta = "/content/drive/MyDrive/Colab Notebooks/Corpus/task1MODneuronersm.txt"

# Número de tokens

El número de tokens es igual al número de filas una vez hemos tokenizado con spaCy

In [24]:
# Cargamos el archivo de train de la Tarea 1
# Si no ponemos on_bad_lines = "skip" obtenemos diferente valor en 876191	en	casos_clinicos_profesiones91	536	538	B-PROFESION
# Si no ponemos quoting = 3 obtenemos varios errores, por ejemplo, S0210-48062003000100010-1 917 918 O\nsecond S...	
train = pd.read_csv(ruta, sep=' ', 
                    names=['words', 'fileId', 'start', 'end', 'label'], quoting=3, on_bad_lines='skip')

# Transformamos las variables label y words
train['label'] = train['label'].astype('category')
train['words'] = train['words'].astype('str')
train

,words,fileId,start,end,label
0,El,32085846_ES,0,3,O
1,21,32085846_ES,4,6,O
2,de,32085846_ES,7,9,O
3,enero,32085846_ES,10,15,O
4,de,32085846_ES,16,18,O
...,...,...,...,...,...
223886,residual,es-S0211-57352013000300011-1,4533,4541,O
223887,e,es-S0211-57352013000300011-1,4542,4543,O
223888,insight,es-S0211-57352013000300011-1,4544,4551,O
223889,parcial,es-S0211-57352013000300011-1,4552,4559,O


### Número de tokens después de aplicar NEURONER

El número de tokens es igual al número de filas una vez hemos tokenizado con spaCy

In [9]:
# Número de tokens en el conjunto de train
len(train.words)

223891

# Número de sentences y caracteres

In [10]:
train['sentence'] = train.groupby(['fileId'])['words'].transform(lambda x: ' '.join(x))
train['word_labels'] = train.groupby(['fileId'])['label'].transform(lambda x: ' '.join(x))
train

,words,fileId,start,end,label,sentence,word_labels
0,El,32085846_ES,0,3,O,"El 21 de enero de 2020 , un hombre de 50 años ...",O O O O O O O O O O O O O O O O O O O O O O O ...
1,21,32085846_ES,4,6,O,"El 21 de enero de 2020 , un hombre de 50 años ...",O O O O O O O O O O O O O O O O O O O O O O O ...
2,de,32085846_ES,7,9,O,"El 21 de enero de 2020 , un hombre de 50 años ...",O O O O O O O O O O O O O O O O O O O O O O O ...
3,enero,32085846_ES,10,15,O,"El 21 de enero de 2020 , un hombre de 50 años ...",O O O O O O O O O O O O O O O O O O O O O O O ...
4,de,32085846_ES,16,18,O,"El 21 de enero de 2020 , un hombre de 50 años ...",O O O O O O O O O O O O O O O O O O O O O O O ...
...,...,...,...,...,...,...,...
223886,residual,es-S0211-57352013000300011-1,4533,4541,O,Antecedentes y aparición del Delirio de Capgra...,O O O O O O O O O O O O O O O O O O B-SITUACIO...
223887,e,es-S0211-57352013000300011-1,4542,4543,O,Antecedentes y aparición del Delirio de Capgra...,O O O O O O O O O O O O O O O O O O B-SITUACIO...
223888,insight,es-S0211-57352013000300011-1,4544,4551,O,Antecedentes y aparición del Delirio de Capgra...,O O O O O O O O O O O O O O O O O O B-SITUACIO...
223889,parcial,es-S0211-57352013000300011-1,4552,4559,O,Antecedentes y aparición del Delirio de Capgra...,O O O O O O O O O O O O O O O O O O B-SITUACIO...


In [11]:
data_train

,texto,fichero
1,"﻿El 21 de enero de 2020, un hombre de 50 años ...",32085846_ES
3,"El 2 de febrero de 2020, una mujer de 28 años ...",32119083_ES
5,Ingresó en nuestro hospital un hombre de 23 añ...,32168162_ES
7,Nuestro paciente es un hombre de 56 años con i...,32222713_ES
9,"﻿Una niña de 5 semanas, nacida a término, con ...",32318729_ES
...,...,...
521,"Apicultor de 58 años, sin antecedentes de inte...",cc_reumatologia52
523,Hombre de 57 años que acude al servicio de urg...,cc_reumatologia76
525,Paciente de 35 años que acude a urgencias porq...,cc_reumatologia77
527,Varón de 83 años de edad con antecedentes pers...,cc_reumatologia88


In [12]:
# Recuento de frases
# https://stackoverflow.com/questions/63919115/count-how-many-sentences-there-are-in-each-row-within-a-pandas-column
!pip install textstat
import textstat

# Cuando aparecen número decimales falla (lo divide como una sentencia)
data_train['sentences'] = data_train['texto'].str.count('[\w][\.!\?]').clip(lower=1)
data_train['sentences_textstat'] = data_train['texto'].apply(textstat.sentence_count)
data_train['caracteres'] = data_train['texto'].str.len()
# Esto no se hace así, se hace con el tokenizador
# data_train["tokens"] = data_train.texto.str.count(' ')

# Recuento de tokens
# Importamos el tokenizador de spaCy
!pip install spacy
!python -m spacy download es_core_news_sm

import spacy
nlp = spacy.load("es_core_news_sm")
data_train['texto_tokenizado'] = data_train['texto'].apply(lambda x: nlp(x))
data_train["tokens"] = data_train.texto_tokenizado.agg([len])

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


<ipython-input-12-535d66a99ca8>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train['sentences'] = data_train['texto'].str.count('[\w][\.!\?]').clip(lower=1)
<ipython-input-12-535d66a99ca8>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train['sentences_textstat'] = data_train['texto'].apply(textstat.sentence_count)
<ipython-input-12-535d66a99ca8>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-01-29 19:29:23.868738: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 36.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
<ipython-input-12-535d66a99ca8>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train['texto_tokenizado'] = data_train['texto'].apply(lambda x: nlp(x))
<ipython-input-12-535d66a99ca8>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train["tokens"] = data_train.texto_tokenizado.agg([len])


In [13]:
data_train

,texto,fichero,sentences,sentences_textstat,caracteres,texto_tokenizado,tokens
1,"﻿El 21 de enero de 2020, un hombre de 50 años ...",32085846_ES,22,24,3176,"(﻿El, 21, de, enero, de, 2020, ,, un, hombre, ...",573
3,"El 2 de febrero de 2020, una mujer de 28 años ...",32119083_ES,33,29,4896,"(El, 2, de, febrero, de, 2020, ,, una, mujer, ...",915
5,Ingresó en nuestro hospital un hombre de 23 añ...,32168162_ES,12,16,2162,"(Ingresó, en, nuestro, hospital, un, hombre, d...",398
7,Nuestro paciente es un hombre de 56 años con i...,32222713_ES,51,53,7913,"(Nuestro, paciente, es, un, hombre, de, 56, añ...",1329
9,"﻿Una niña de 5 semanas, nacida a término, con ...",32318729_ES,35,33,4393,"(﻿Una, niña, de, 5, semanas, ,, nacida, a, tér...",802
...,...,...,...,...,...,...,...
521,"Apicultor de 58 años, sin antecedentes de inte...",cc_reumatologia52,9,10,1703,"(Apicultor, de, 58, años, ,, sin, antecedentes...",301
523,Hombre de 57 años que acude al servicio de urg...,cc_reumatologia76,46,43,2743,"(Hombre, de, 57, años, que, acude, al, servici...",559
525,Paciente de 35 años que acude a urgencias porq...,cc_reumatologia77,39,36,3316,"(Paciente, de, 35, años, que, acude, a, urgenc...",574
527,Varón de 83 años de edad con antecedentes pers...,cc_reumatologia88,9,10,1456,"(Varón, de, 83, años, de, edad, con, anteceden...",240


In [14]:
# Número medio de sentence train
print("Sentences train:")
print("Número medio de sentences train: ", data_train.sentences.mean())
print("Número total de sentences train: ", data_train.sentences.sum())
print("Número mínimo de sentences train: ", data_train.sentences.min())
print("Número máximo de sentences train: ", data_train.sentences.max())
print(" ")
print("sentences_textstat train:")
print("Número medio de sentences_textstat train: ", data_train.sentences_textstat.mean())
print("Número total de sentences_textstat train: ", data_train.sentences_textstat.sum())
print("Número mínimo de sentences_textstat train: ", data_train.sentences_textstat.min())
print("Número máximo de sentences_textstat train: ", data_train.sentences_textstat.max())
print(" ")
print("Caracteres train:")
print("Número medio de caracteres train: ", data_train.caracteres.mean())
print("Número total de caracteres train: ", data_train.caracteres.sum())
print("Número mínimo de caracteres train: ", data_train.caracteres.min())
print("Número máximo de caracteres train: ", data_train.caracteres.max())
print(" ")

print("Tokens train:")
print("Número medio de tokens train: ", data_train.tokens.mean())
print("Número total de tokens train: ", data_train.tokens.sum())
print("Número mínimo de tokens train: ", data_train.tokens.min())
print("Número máximo de tokens train: ", data_train.tokens.max())

Sentences train:
Número medio de sentences train:  36.77735849056604
Número total de sentences train:  9746
Número mínimo de sentences train:  5
Número máximo de sentences train:  230
 
sentences_textstat train:
Número medio de sentences_textstat train:  37.41132075471698
Número total de sentences_textstat train:  9914
Número mínimo de sentences_textstat train:  5
Número máximo de sentences_textstat train:  202
 
Caracteres train:
Número medio de caracteres train:  4879.28679245283
Número total de caracteres train:  1293011
Número mínimo de caracteres train:  567
Número máximo de caracteres train:  25076
 
Tokens train:
Número medio de tokens train:  861.6075471698114
Número total de tokens train:  228326
Número mínimo de tokens train:  95
Número máximo de tokens train:  4720


# Número de entidades

Vemos todas las etiquetas que hay

In [ ]:
train.label.value_counts()

O                      222265
I-PROFESION               604
B-PROFESION               420
I-SITUACION_LABORAL       308
B-SITUACION_LABORAL       175
I-ACTIVIDAD                80
B-ACTIVIDAD                39
Name: label, dtype: int64

In [ ]:
# Train
# Nos quedamos con fila única
entidades_train = train[["fileId", "word_labels"]].drop_duplicates()
# Contamos cuantas veces aparecen las etiquetas en cada fila
entidades_train["nentidades"] = entidades_train.word_labels.str.count("B-PROFESION|I-PROFESION|B-SITUACION_LABORAL|I-SITUACION_LABORAL|B-ACTIVIDAD|I-ACTIVIDAD")

In [ ]:
entidades_train

,fileId,word_labels,nentidades
0,32085846_ES,O O O O O O O O O O O O O O O O O O O O O O O ...,2
571,32119083_ES,O O O O O O O O O O O O O O O O O O O O O O O ...,2
1483,32168162_ES,O O O O O O O O O O O O O O O O O O O O O O O ...,3
1878,32222713_ES,O O O O O O O O O O O O O O O O O O O O O O O ...,3
3199,32318729_ES,O O O O O O O O O O O O O O O O O O O O O O O ...,11
...,...,...,...
221516,cc_reumatologia52,B-PROFESION O O O O O O O O O O O O O O O O O ...,1
221817,cc_reumatologia76,O O O O O O O O O O O O O O O O O O O O O O O ...,2
222330,cc_reumatologia77,O O O O O O O O O O O O O O O O O O O O O O O ...,6
222876,cc_reumatologia88,O O O O O O O O O O O O O O O O O O O O O O O ...,4


In [ ]:
# Sacamos el resto de métricas
print("Número entidades totales train:", entidades_train.nentidades.sum())
print("Número entidades medias train:", entidades_train.nentidades.mean())
print("Número entidades medias train (de las frases con entidades):", entidades_train[entidades_train.nentidades >0].nentidades.mean())
print("Número entidades minimas train (de las frases con entidades):", entidades_train[entidades_train.nentidades >0].nentidades.min())
print("Número entidades maximas train (de las frases con entidades):", entidades_train[entidades_train.nentidades >0].nentidades.max())

Número entidades totales train: 1626
Número entidades medias train: 6.1358490566037736
Número entidades medias train (de las frases con entidades): 6.1358490566037736
Número entidades minimas train (de las frases con entidades): 1
Número entidades maximas train (de las frases con entidades): 75


# Otras métricas

Cargamos cuatro archivos, 2 por cada tarea (ner / class) y 2 por cada conjunto (train / test)

Estos archivos los obtenemos haciendo lo siguiente:

1. Identificamos los segmentos del tipo: PROFESION 75 93 , con \t.*\t seleccionamos todo el texto, lo copiamos a otra hoja y sustituimos los espacios por tabulaciones
2. Una vez hecho esto seleccionamos los segmentos hasta el primer espacio “.* “: T1	PROFESION 75 93	médico de cabecera; con esto lo copiamos a otra hoja, cambiamos el espacio final lo cambiamos por tabulador y lo devolvemos a la hoja original

Sustituimos por \t

Para ver las instancias malas nos quedamos con el inicio y con el documento


`awk '{print $0,FILENAME}' *.ann >> ~/Desktop/profnervalid.txt`

EL OBJETIVO ES CONSEGUIR AGRUPAR TODOS LOS ARCHIVOS .ANN EN UN ÚNICO ARCHIVO .TXT

In [ ]:
task1train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Corpus/task1MOD.txt", sep = "\t", names = ["Tag", "Entity", "start", "end", "freetext", "document"], header = None)
task2train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Corpus/task2MOD.txt", sep = "\t", names = ["Tag", "Entity", "start", "end", "freetext", "document"], header = None)

In [ ]:
task1train

,Tag,Entity,start,end,freetext,document
0,T1,SITUACION_LABORAL,352,369,siguió trabajando,32085846_ES.ann
1,T1,PROFESION,4689,4713,profesionales sanitarios,32119083_ES.ann
2,T1,SITUACION_LABORAL,628,651,había estado trabajando,32168162_ES.ann
3,T1,SITUACION_LABORAL,723,744,profesional en activo,32222713_ES.ann
4,T1,PROFESION,915,978,trabaja en una instalación de atención a clien...,32318729_ES.ann
...,...,...,...,...,...,...
634,T1,SITUACION_LABORAL,105,135,licenciada en ciencias exactas,es-S0211-57352013000300011-1.ann
635,T3,PROFESION,138,163,ex profesora de instituto,es-S0211-57352013000300011-1.ann
636,T5,PROFESION,1954,1962,profesor,es-S0211-57352013000300011-1.ann
637,T7,PROFESION,1832,1840,profesor,es-S0211-57352013000300011-1.ann


In [ ]:
# Número de palabras
task1train["n_palabras"] = task1train.freetext.str.count(' ')
task2train["n_palabras"] = task2train.freetext.str.count(' ')

In [ ]:
task1train

,Tag,Entity,start,end,freetext,document,n_palabras
0,T1,SITUACION_LABORAL,352,369,siguió trabajando,32085846_ES.ann,1
1,T1,PROFESION,4689,4713,profesionales sanitarios,32119083_ES.ann,1
2,T1,SITUACION_LABORAL,628,651,había estado trabajando,32168162_ES.ann,2
3,T1,SITUACION_LABORAL,723,744,profesional en activo,32222713_ES.ann,2
4,T1,PROFESION,915,978,trabaja en una instalación de atención a clien...,32318729_ES.ann,9
...,...,...,...,...,...,...,...
634,T1,SITUACION_LABORAL,105,135,licenciada en ciencias exactas,es-S0211-57352013000300011-1.ann,3
635,T3,PROFESION,138,163,ex profesora de instituto,es-S0211-57352013000300011-1.ann,3
636,T5,PROFESION,1954,1962,profesor,es-S0211-57352013000300011-1.ann,0
637,T7,PROFESION,1832,1840,profesor,es-S0211-57352013000300011-1.ann,0


In [ ]:
task1train.Entity.value_counts()

PROFESION            421
SITUACION_LABORAL    178
ACTIVIDAD             40
Name: Entity, dtype: int64

In [ ]:
task2train.Entity.value_counts()

PACIENTE     359
SANITARIO    185
OTROS         54
FAMILIAR      41
Name: Entity, dtype: int64

In [ ]:
# Vemos distribución de etiquetas entre NER y CLASS del entrenamiento y test
task1_task2_train = pd.merge(task1train,task2train,on=["document", "start", "end", "freetext", "n_palabras"])
task1_task2_train

,Tag_x,Entity_x,start,end,freetext,document,n_palabras,Tag_y,Entity_y
0,T1,SITUACION_LABORAL,352,369,siguió trabajando,32085846_ES.ann,1,T2,PACIENTE
1,T1,PROFESION,4689,4713,profesionales sanitarios,32119083_ES.ann,1,T2,SANITARIO
2,T1,SITUACION_LABORAL,628,651,había estado trabajando,32168162_ES.ann,2,T2,PACIENTE
3,T1,SITUACION_LABORAL,723,744,profesional en activo,32222713_ES.ann,2,T2,PACIENTE
4,T1,PROFESION,915,978,trabaja en una instalación de atención a clien...,32318729_ES.ann,9,T2,FAMILIAR
...,...,...,...,...,...,...,...,...,...
634,T1,SITUACION_LABORAL,105,135,licenciada en ciencias exactas,es-S0211-57352013000300011-1.ann,3,T2,PACIENTE
635,T3,PROFESION,138,163,ex profesora de instituto,es-S0211-57352013000300011-1.ann,3,T4,PACIENTE
636,T5,PROFESION,1954,1962,profesor,es-S0211-57352013000300011-1.ann,0,T6,OTROS
637,T7,PROFESION,1832,1840,profesor,es-S0211-57352013000300011-1.ann,0,T8,OTROS


In [ ]:
task1_task2_train.value_counts(["Entity_x", "Entity_y"])

Entity_x           Entity_y 
PROFESION          PACIENTE     186
                   SANITARIO    185
SITUACION_LABORAL  PACIENTE     133
PROFESION          OTROS         41
ACTIVIDAD          PACIENTE      40
SITUACION_LABORAL  FAMILIAR      32
                   OTROS         13
PROFESION          FAMILIAR       9
dtype: int64

In [ ]:
# Encontramos los errores
comprobacion = task1_task2_train[["document", "start", "end"]]
comprobacion[comprobacion.duplicated(["document", "start", "end"], keep=False)]

,document,start,end
